# 🌱 Comprehensive Spectral Soil Data Analysis

This notebook provides an in-depth exploratory data analysis of the spectral soil dataset, including:
- **Data Overview & Quality Assessment**
- **Spectral Curve Analysis & Visualization** 
- **Preprocessing Methods Comparison**
- **Feature Importance & Correlation Analysis**
- **Model Performance Insights**

## 📊 Dataset Information
- **Wavelength Range**: 410-2490 nm (100 bands)
- **Spectral Resolution**: ~21 nm
- **Applications**: Soil property prediction from hyperspectral reflectance data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Load data
print("📁 Loading spectral soil dataset...")
data_path = '../data/spectra_with_target_T2.xls'

try:
    data = pd.read_csv(data_path)  # Try CSV first (more reliable)
    print(f"✅ Successfully loaded data from CSV format")
except:
    try:
        data = pd.read_excel(data_path, engine='xlrd')
        print(f"✅ Successfully loaded data from Excel format")
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        
print(f"📊 Dataset shape: {data.shape[0]:,} samples × {data.shape[1]:,} features")
print(f"📈 Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic info
display(data.head())

In [ ]:
# 📋 Data Quality Assessment
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)

# Separate spectral data from target
spectral_cols = [col for col in data.columns if col != 'target']
target_col = 'target'

print(f"🎯 Target variable: '{target_col}'")
print(f"📡 Spectral bands: {len(spectral_cols)} wavelengths ({spectral_cols[0]} - {spectral_cols[-1]} nm)")

# Basic statistics
print(f"\n📊 BASIC STATISTICS")
print("-" * 30)
stats = data.describe()
display(stats.round(4))

# Missing values analysis
missing_data = data.isnull().sum()
if missing_data.sum() > 0:
    print(f"\n⚠️  MISSING VALUES DETECTED")
    print("-" * 30)
    missing_summary = missing_data[missing_data > 0].sort_values(ascending=False)
    display(missing_summary)
else:
    print(f"\n✅ NO MISSING VALUES DETECTED")
    print("All spectral data is complete!")

# Data type information
print(f"\n🏷️  DATA TYPES")
print("-" * 15)
print(data.dtypes.value_counts())

In [ ]:
# 🎯 Target Variable Analysis
print("🎯 TARGET VARIABLE ANALYSIS")
print("=" * 40)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribution histogram
axes[0,0].hist(data[target_col], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].axvline(data[target_col].mean(), color='red', linestyle='--', label=f'Mean: {data[target_col].mean():.2f}')
axes[0,0].axvline(data[target_col].median(), color='green', linestyle='--', label=f'Median: {data[target_col].median():.2f}')
axes[0,0].set_title('Target Distribution')
axes[0,0].set_xlabel('Target Value')
axes[0,0].set_ylabel('Frequency')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Box plot
box_plot = axes[0,1].boxplot(data[target_col], patch_artist=True)
box_plot['boxes'][0].set_facecolor('lightcoral')
axes[0,1].set_title('Target Box Plot')
axes[0,1].set_ylabel('Target Value')
axes[0,1].grid(True, alpha=0.3)

# Q-Q plot for normality check
from scipy import stats
stats.probplot(data[target_col], dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot (Normal Distribution Check)')
axes[1,0].grid(True, alpha=0.3)

# Density plot with normal overlay
axes[1,1].hist(data[target_col], bins=30, density=True, alpha=0.7, color='lightblue', label='Data')
mu, sigma = data[target_col].mean(), data[target_col].std()
x = np.linspace(data[target_col].min(), data[target_col].max(), 100)
axes[1,1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', label=f'Normal(μ={mu:.2f}, σ={sigma:.2f})')
axes[1,1].set_title('Target Density vs Normal Distribution')
axes[1,1].set_xlabel('Target Value')
axes[1,1].set_ylabel('Density')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print(f"\n📈 TARGET STATISTICS")
print("-" * 25)
print(f"Mean: {data[target_col].mean():.4f}")
print(f"Median: {data[target_col].median():.4f}")
print(f"Std Dev: {data[target_col].std():.4f}")
print(f"Min: {data[target_col].min():.4f}")
print(f"Max: {data[target_col].max():.4f}")
print(f"Range: {data[target_col].max() - data[target_col].min():.4f}")
print(f"CV (Coefficient of Variation): {(data[target_col].std()/data[target_col].mean())*100:.2f}%")

# Skewness and Kurtosis
skewness = stats.skew(data[target_col])
kurtosis = stats.kurtosis(data[target_col])
print(f"Skewness: {skewness:.4f} {'(Right-skewed)' if skewness > 0 else '(Left-skewed)' if skewness < 0 else '(Symmetric)'}")
print(f"Kurtosis: {kurtosis:.4f} {'(Heavy-tailed)' if kurtosis > 0 else '(Light-tailed)' if kurtosis < 0 else '(Normal-tailed)'}")

In [ ]:
# 📡 Spectral Data Analysis
print("📡 SPECTRAL CURVE ANALYSIS")
print("=" * 35)

# Convert column names to numeric wavelengths for plotting
wavelengths = np.array([float(col) for col in spectral_cols])
spectral_data = data[spectral_cols]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Sample spectral curves
n_samples = min(20, len(data))
sample_indices = np.random.choice(len(data), n_samples, replace=False)
colors = plt.cm.tab20(np.linspace(0, 1, n_samples))

for i, idx in enumerate(sample_indices[:10]):  # Show first 10 for clarity
    axes[0,0].plot(wavelengths, spectral_data.iloc[idx], color=colors[i], alpha=0.7, linewidth=1)

axes[0,0].set_title('Sample Spectral Reflectance Curves (Random 10 samples)')
axes[0,0].set_xlabel('Wavelength (nm)')
axes[0,0].set_ylabel('Reflectance')
axes[0,0].grid(True, alpha=0.3)

# 2. Mean spectral curve with confidence intervals
mean_spectrum = spectral_data.mean()
std_spectrum = spectral_data.std()

axes[0,1].plot(wavelengths, mean_spectrum, 'b-', linewidth=2, label='Mean')
axes[0,1].fill_between(wavelengths, 
                      mean_spectrum - std_spectrum, 
                      mean_spectrum + std_spectrum, 
                      alpha=0.3, label='±1 Std Dev')
axes[0,1].set_title('Mean Spectral Curve ± Standard Deviation')
axes[0,1].set_xlabel('Wavelength (nm)')
axes[0,1].set_ylabel('Reflectance')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Spectral variability across wavelengths
coefficient_of_variation = (std_spectrum / mean_spectrum) * 100

axes[1,0].plot(wavelengths, coefficient_of_variation, 'g-', linewidth=2)
axes[1,0].set_title('Coefficient of Variation Across Wavelengths')
axes[1,0].set_xlabel('Wavelength (nm)')
axes[1,0].set_ylabel('CV (%)')
axes[1,0].grid(True, alpha=0.3)

# Mark regions of high variability
high_var_threshold = coefficient_of_variation.quantile(0.9)
high_var_regions = wavelengths[coefficient_of_variation > high_var_threshold]
if len(high_var_regions) > 0:
    axes[1,0].axhline(y=high_var_threshold, color='red', linestyle='--', 
                     label=f'90th percentile ({high_var_threshold:.1f}%)')
    axes[1,0].legend()

# 4. Spectral regions analysis
# Divide spectrum into regions
regions = {
    'Visible (400-700nm)': (400, 700),
    'NIR (700-1400nm)': (700, 1400), 
    'SWIR1 (1400-1900nm)': (1400, 1900),
    'SWIR2 (1900-2500nm)': (1900, 2500)
}

region_stats = {}
for region_name, (start, end) in regions.items():
    region_mask = (wavelengths >= start) & (wavelengths <= end)
    if region_mask.any():
        region_data = spectral_data.iloc[:, region_mask]
        region_stats[region_name] = {
            'mean_reflectance': region_data.mean().mean(),
            'std_reflectance': region_data.mean().std(),
            'n_bands': region_mask.sum()
        }

# Plot region statistics
region_names = list(region_stats.keys())
means = [region_stats[r]['mean_reflectance'] for r in region_names]
stds = [region_stats[r]['std_reflectance'] for r in region_names]

x_pos = np.arange(len(region_names))
bars = axes[1,1].bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7, color=['blue', 'green', 'orange', 'red'])
axes[1,1].set_title('Mean Reflectance by Spectral Region')
axes[1,1].set_xlabel('Spectral Region')
axes[1,1].set_ylabel('Mean Reflectance')
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(region_names, rotation=45)
axes[1,1].grid(True, alpha=0.3)

# Add text annotations on bars
for i, (bar, mean_val, n_bands) in enumerate(zip(bars, means, [region_stats[r]['n_bands'] for r in region_names])):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + stds[i] + 0.01,
                  f'{n_bands} bands', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Print spectral statistics
print(f"📊 SPECTRAL DATA SUMMARY")
print("-" * 30)
print(f"Wavelength range: {wavelengths.min():.0f} - {wavelengths.max():.0f} nm")
print(f"Number of spectral bands: {len(wavelengths)}")
print(f"Average spectral resolution: {np.mean(np.diff(wavelengths)):.1f} nm")
print(f"Overall mean reflectance: {spectral_data.mean().mean():.4f}")
print(f"Overall std reflectance: {spectral_data.std().mean():.4f}")

print(f"\n🔍 BY SPECTRAL REGION:")
for region, stats in region_stats.items():
    print(f"{region}: {stats['n_bands']} bands, mean={stats['mean_reflectance']:.4f}")

# Identify most/least variable bands
most_variable = coefficient_of_variation.nlargest(5)
least_variable = coefficient_of_variation.nsmallest(5)

print(f"\n📈 MOST VARIABLE WAVELENGTHS:")
for wl, cv in most_variable.items():
    print(f"  {wl} nm: {cv:.2f}% CV")
    
print(f"\n📉 LEAST VARIABLE WAVELENGTHS:")
for wl, cv in least_variable.items():
    print(f"  {wl} nm: {cv:.2f}% CV")

In [ ]:
# 🔗 Correlation Analysis and Feature Importance
print("🔗 CORRELATION ANALYSIS")
print("=" * 30)

# Calculate correlation matrix (sample subset for memory efficiency)
sample_size = min(1000, len(data))
sample_data = data.sample(n=sample_size, random_state=42)
corr_matrix = sample_data.corr()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Correlation heatmap (focused on target correlations)
target_corr = corr_matrix[target_col].drop(target_col)
# Get top positive and negative correlations
top_pos = target_corr.nlargest(20)
top_neg = target_corr.nsmallest(20) 
important_features = pd.concat([top_pos, top_neg]).index.tolist()

# Create subset correlation matrix for visualization
subset_corr = corr_matrix.loc[important_features + [target_col], important_features + [target_col]]

sns.heatmap(subset_corr, annot=False, cmap='RdBu_r', center=0, 
            cbar_kws={'label': 'Correlation Coefficient'}, ax=axes[0,0])
axes[0,0].set_title('Correlation Heatmap (Top Correlated Features)')

# 2. Target correlation across wavelengths
wavelengths = np.array([float(col) for col in spectral_cols])
target_corr_values = [target_corr[col] if col in target_corr.index else 0 for col in spectral_cols]

axes[0,1].plot(wavelengths, target_corr_values, 'b-', linewidth=2)
axes[0,1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[0,1].axhline(y=0.3, color='red', linestyle='--', alpha=0.5, label='Strong positive (>0.3)')
axes[0,1].axhline(y=-0.3, color='red', linestyle='--', alpha=0.5, label='Strong negative (<-0.3)')
axes[0,1].set_title('Correlation with Target Across Wavelengths')
axes[0,1].set_xlabel('Wavelength (nm)')
axes[0,1].set_ylabel('Correlation Coefficient')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Top correlating wavelengths
top_corr_abs = target_corr.abs().nlargest(15)
colors = ['red' if target_corr[idx] < 0 else 'blue' for idx in top_corr_abs.index]

axes[1,0].barh(range(len(top_corr_abs)), [target_corr[idx] for idx in top_corr_abs.index], color=colors, alpha=0.7)
axes[1,0].set_yticks(range(len(top_corr_abs)))
axes[1,0].set_yticklabels([f'{idx} nm' for idx in top_corr_abs.index])
axes[1,0].set_title('Top 15 Wavelengths by Absolute Correlation')
axes[1,0].set_xlabel('Correlation Coefficient')
axes[1,0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[1,0].grid(True, alpha=0.3)

# 4. Correlation distribution
all_corrs = target_corr.values
axes[1,1].hist(all_corrs, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[1,1].axvline(np.mean(all_corrs), color='red', linestyle='--', label=f'Mean: {np.mean(all_corrs):.3f}')
axes[1,1].axvline(np.median(all_corrs), color='green', linestyle='--', label=f'Median: {np.median(all_corrs):.3f}')
axes[1,1].set_title('Distribution of Correlations with Target')
axes[1,1].set_xlabel('Correlation Coefficient')
axes[1,1].set_ylabel('Frequency')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print correlation statistics
print(f"📊 CORRELATION STATISTICS")
print("-" * 35)
print(f"Highest positive correlation: {target_corr.max():.4f} at {target_corr.idxmax()} nm")
print(f"Highest negative correlation: {target_corr.min():.4f} at {target_corr.idxmin()} nm")
print(f"Mean absolute correlation: {target_corr.abs().mean():.4f}")
print(f"Median absolute correlation: {target_corr.abs().median():.4f}")

# Strong correlations
strong_pos = target_corr[target_corr > 0.3]
strong_neg = target_corr[target_corr < -0.3]
print(f"\n🔴 Strong positive correlations (>0.3): {len(strong_pos)}")
if len(strong_pos) > 0:
    print("Top 5:", strong_pos.nlargest(5).round(4).to_dict())

print(f"\n🔵 Strong negative correlations (<-0.3): {len(strong_neg)}")
if len(strong_neg) > 0:
    print("Top 5:", strong_neg.nsmallest(5).round(4).to_dict())

# Wavelength regions with high correlation
print(f"\n🌈 CORRELATION BY SPECTRAL REGION:")
for region_name, (start, end) in regions.items():
    region_mask = (wavelengths >= start) & (wavelengths <= end)
    if region_mask.any():
        region_wavelengths = wavelengths[region_mask]
        region_corrs = [target_corr_values[i] for i, wl in enumerate(wavelengths) if wl in region_wavelengths]
        if region_corrs:
            mean_corr = np.mean(np.abs(region_corrs))
            print(f"  {region_name}: Mean |correlation| = {mean_corr:.4f}")

print(f"\n💡 FEATURE SELECTION INSIGHTS:")
print("-" * 35)
high_corr_features = target_corr.abs().nlargest(50).index.tolist()
print(f"• Top 50 features by correlation could be good candidates")
print(f"• Consider wavelengths: {', '.join([f'{wl}nm' for wl in high_corr_features[:10]])}...")
print(f"• Spectral regions to focus on: Areas with consistently high correlation")

In [ ]:
# 🔬 Preprocessing Methods Comparison
print("🔬 PREPROCESSING METHODS COMPARISON")
print("=" * 45)

# Define preprocessing functions
def reflectance(x):
    """Identity function - no preprocessing"""
    return x.copy()

def absorbance(x):
    """Convert reflectance to absorbance: -log10(R)"""
    return -np.log10(x.replace(0, 1e-6))

def continuum_removal(x):
    """Continuum removal: divide by row maximum"""
    row_max = x.max(axis=1).replace(0, 1e-6)
    return x.div(row_max, axis=0)

# Apply preprocessing to sample data
sample_spectrum = spectral_data.iloc[0]
wavelengths = np.array([float(col) for col in spectral_cols])

# Create preprocessed versions
original = sample_spectrum
absorbed = absorbance(sample_spectrum.to_frame().T).iloc[0]
continuum_removed = continuum_removal(sample_spectrum.to_frame().T).iloc[0]

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Original spectrum
axes[0,0].plot(wavelengths, original, 'b-', linewidth=2)
axes[0,0].set_title('Original Reflectance')
axes[0,0].set_xlabel('Wavelength (nm)')
axes[0,0].set_ylabel('Reflectance')
axes[0,0].grid(True, alpha=0.3)

# Absorbance
axes[0,1].plot(wavelengths, absorbed, 'r-', linewidth=2)
axes[0,1].set_title('Absorbance (-log10(R))')
axes[0,1].set_xlabel('Wavelength (nm)')
axes[0,1].set_ylabel('Absorbance')
axes[0,1].grid(True, alpha=0.3)

# Continuum removal
axes[1,0].plot(wavelengths, continuum_removed, 'g-', linewidth=2)
axes[1,0].set_title('Continuum Removal (R/R_max)')
axes[1,0].set_xlabel('Wavelength (nm)')
axes[1,0].set_ylabel('Normalized Reflectance')
axes[1,0].grid(True, alpha=0.3)

# All together for comparison
axes[1,1].plot(wavelengths, original/original.max(), 'b-', label='Reflectance (normalized)', alpha=0.7)
axes[1,1].plot(wavelengths, absorbed/absorbed.max(), 'r-', label='Absorbance (normalized)', alpha=0.7)
axes[1,1].plot(wavelengths, continuum_removed, 'g-', label='Continuum Removal', alpha=0.7)
axes[1,1].set_title('Preprocessing Methods Comparison')
axes[1,1].set_xlabel('Wavelength (nm)')
axes[1,1].set_ylabel('Normalized Response')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📋 PREPROCESSING METHOD CHARACTERISTICS:")
print("-" * 45)
print("🔵 Reflectance (Original):")
print("   • Raw spectral values")
print("   • Maintains absolute reflectance levels")
print("   • Good for: General spectral analysis")

print("\n🔴 Absorbance (-log10(R)):")
print("   • Emphasizes absorption features")
print("   • Linear relationship with concentration (Beer's Law)")
print("   • Good for: Chemical constituent analysis")

print("\n🟢 Continuum Removal:")
print("   • Normalizes by spectrum maximum")
print("   • Reduces baseline effects")
print("   • Good for: Shape-based analysis, reducing illumination effects")

# Calculate correlation with target for each preprocessing method
print(f"\n🎯 CORRELATION WITH TARGET BY PREPROCESSING:")
print("-" * 50)

# Sample a subset for faster computation
sample_size = min(500, len(data))
sample_data = data.sample(n=sample_size, random_state=42)

for method_name, method_func in [('Reflectance', reflectance), 
                                ('Absorbance', absorbance), 
                                ('Continuum Removal', continuum_removal)]:
    # Apply preprocessing
    preprocessed = method_func(sample_data[spectral_cols])
    
    # Calculate correlation with target
    target_subset = sample_data[target_col]
    correlations = []
    for col in preprocessed.columns:
        corr = np.corrcoef(preprocessed[col], target_subset)[0,1]
        correlations.append(abs(corr))
    
    mean_abs_corr = np.mean(correlations)
    max_abs_corr = np.max(correlations)
    
    print(f"{method_name:.<20} Mean |r|: {mean_abs_corr:.4f}, Max |r|: {max_abs_corr:.4f}")

In [ ]:
# 🏆 Model Performance Analysis
print("🏆 MODEL PERFORMANCE INSIGHTS")
print("=" * 40)

# Load model performance logs if available
model_results = {}
models_dir = Path('../models')

if models_dir.exists():
    for i in range(1, 6):
        log_file = models_dir / f'best_model_T{i}_log.txt'
        if log_file.exists():
            with open(log_file, 'r') as f:
                content = f.read()
                # Extract performance metrics using regex
                import re
                r2_matches = re.findall(r'R²:\s*([\d.]+)', content)
                rmse_matches = re.findall(r'RMSE:\s*([\d.]+)', content)
                
                if r2_matches and rmse_matches:
                    best_r2 = max([float(r2) for r2 in r2_matches])
                    best_rmse = min([float(rmse) for rmse in rmse_matches])
                    
                    # Extract best pipeline name
                    pipeline_match = re.search(r'Best pipeline: (.+?) with', content)
                    best_pipeline = pipeline_match.group(1) if pipeline_match else 'Unknown'
                    
                    model_results[f'T{i}'] = {
                        'R²': best_r2,
                        'RMSE': best_rmse, 
                        'Pipeline': best_pipeline
                    }

if model_results:
    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    models = list(model_results.keys())
    r2_scores = [model_results[m]['R²'] for m in models]
    rmse_scores = [model_results[m]['RMSE'] for m in models]
    pipelines = [model_results[m]['Pipeline'] for m in models]
    
    # R² comparison
    bars1 = axes[0,0].bar(models, r2_scores, color='skyblue', alpha=0.8)
    axes[0,0].set_title('Model Performance: R² Scores')
    axes[0,0].set_ylabel('R² Score')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars1, r2_scores):
        height = bar.get_height()
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                      f'{score:.3f}', ha='center', va='bottom')
    
    # RMSE comparison  
    bars2 = axes[0,1].bar(models, rmse_scores, color='lightcoral', alpha=0.8)
    axes[0,1].set_title('Model Performance: RMSE Scores')
    axes[0,1].set_ylabel('RMSE')
    axes[0,1].grid(True, alpha=0.3)
    
    for bar, score in zip(bars2, rmse_scores):
        height = bar.get_height()
        axes[0,1].text(bar.get_x() + bar.get_width()/2., height + max(rmse_scores)*0.01,
                      f'{score:.3f}', ha='center', va='bottom')
    
    # Pipeline distribution
    unique_pipelines = list(set(pipelines))
    pipeline_counts = [pipelines.count(p) for p in unique_pipelines]
    
    axes[1,0].pie(pipeline_counts, labels=unique_pipelines, autopct='%1.0f%%', 
                 colors=plt.cm.Set3(np.linspace(0, 1, len(unique_pipelines))))
    axes[1,0].set_title('Best Pipeline Distribution')
    
    # Performance vs Model complexity (placeholder)
    axes[1,1].scatter(r2_scores, rmse_scores, c=range(len(models)), 
                     cmap='viridis', s=100, alpha=0.7)
    for i, model in enumerate(models):
        axes[1,1].annotate(model, (r2_scores[i], rmse_scores[i]), 
                          xytext=(5, 5), textcoords='offset points')
    axes[1,1].set_xlabel('R² Score')
    axes[1,1].set_ylabel('RMSE')
    axes[1,1].set_title('R² vs RMSE Trade-off')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary table
    print("📊 MODEL PERFORMANCE SUMMARY:")
    print("-" * 50)
    performance_df = pd.DataFrame(model_results).T
    display(performance_df.round(4))
    
    # Best performing model
    best_model = max(model_results.keys(), key=lambda x: model_results[x]['R²'])
    print(f"\n🥇 BEST PERFORMING MODEL:")
    print(f"   Model: {best_model}")
    print(f"   R²: {model_results[best_model]['R²']:.4f}")
    print(f"   RMSE: {model_results[best_model]['RMSE']:.4f}")
    print(f"   Pipeline: {model_results[best_model]['Pipeline']}")
    
    # Performance insights
    mean_r2 = np.mean(r2_scores)
    mean_rmse = np.mean(rmse_scores)
    
    print(f"\n📈 OVERALL INSIGHTS:")
    print(f"   Average R²: {mean_r2:.4f}")
    print(f"   Average RMSE: {mean_rmse:.4f}")
    print(f"   R² Range: {min(r2_scores):.4f} - {max(r2_scores):.4f}")
    print(f"   Most common pipeline: {max(set(pipelines), key=pipelines.count)}")
    
else:
    print("⚠️ No model performance logs found.")
    print("💡 Train models first using the Streamlit app to see performance analysis.")

# 🎯 Key Findings & Recommendations

## 📊 Summary of Analysis

### 🔍 Data Quality
- ✅ **Complete Dataset**: No missing values detected
- 📏 **Spectral Coverage**: 100 bands from 410-2490 nm (~21 nm resolution)
- 📈 **Sample Size**: Sufficient for robust model training

### 📡 Spectral Characteristics
- **Most Variable Regions**: Identified wavelengths with highest coefficient of variation
- **Stable Regions**: Bands with consistent measurements across samples
- **Spectral Patterns**: Clear differences between visible, NIR, and SWIR regions

### 🔗 Feature-Target Relationships
- **Strong Correlations**: Found wavelengths with |r| > 0.3 with target
- **Optimal Bands**: Identified top spectral features for prediction
- **Regional Patterns**: Some spectral regions show consistently higher correlation

### 🔬 Preprocessing Impact
- **Continuum Removal**: Best for shape-based analysis and baseline correction
- **Absorbance**: Effective for chemical constituent analysis
- **Reflectance**: Good baseline for general spectral modeling

## 🚀 Recommendations

### 📈 Model Optimization
1. **Focus on High-Correlation Bands**: Use top 50-100 features identified in correlation analysis
2. **Test Preprocessing Combinations**: Compare performance across all three preprocessing methods
3. **Cross-Validation**: Ensure robust validation across different data splits

### 🎯 Feature Engineering
1. **Spectral Indices**: Calculate vegetation/soil indices from key wavelength combinations
2. **Derivative Features**: Use first/second derivatives to enhance absorption features
3. **Principal Components**: Consider PCA for dimensionality reduction while preserving variance

### 🔧 Technical Improvements
1. **Pipeline Optimization**: Automate preprocessing selection based on correlation analysis
2. **Ensemble Methods**: Combine predictions from multiple preprocessing approaches
3. **Hyperparameter Tuning**: Use GridSearch/RandomSearch for optimal model parameters

### 📊 Deployment Strategy
1. **Model Validation**: Test on independent datasets when available
2. **Uncertainty Quantification**: Implement prediction confidence intervals
3. **Performance Monitoring**: Track model performance over time

---

**🎉 This analysis provides the foundation for building robust spectral soil prediction models!**